In [1]:
import csv
import json
from os.path import join
import requests
import time

Path to CSV file (download from https://github.com/swiss-art-research-net/bso-vikus-viewer-data/blob/main/data/images.csv)

In [2]:
imageCSV = '../data/images.csv'

Path to JSON file containing classifications, created with [RunwayML](https://runwayml.com/)

In [3]:
inputFile = '../data/Annotation Group - Images and Colour Bars for BSO Examples.json'

Directory to download images into

In [4]:
fileDir = '../training/data/images/'

Download images

In [5]:
imageURLs = {}
with open(imageCSV, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        imageURLs[row['id'] + '.jpg'] = row['image']

In [6]:
with open(inputFile, 'r') as f:
    data = json.load(f)

In [7]:
maxRetries = 5
for file in data['files'].keys():
    outputFile = join(fileDir, file)
    url = imageURLs[file]
    r = requests.get(url, allow_redirects=True)
    retries = 1
    while not 'image' in r.headers['Content-Type'] and retries <= maxRetries:
        # Try again if no image comes back
        time.sleep(1)
        r = requests.get(url, allow_redirects=True)
        retries += 1

    if retries >= maxRetries:
        print("Could not download", id, url)
    else:
        with open(outputFile, 'wb') as f:
            f.write(r.content)